In [ ]:
# Imports
import os
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
)
import optuna
import joblib
from statistics import mean
import math
import torch

from darts import TimeSeries
from darts.models import BlockRNNModel, NBEATSModel, NHiTSModel, TCNModel, TFTModel
from darts.dataprocessing.transformers import Scaler
from darts.utils.likelihood_models import GaussianLikelihood

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable


import warnings

warnings.filterwarnings("ignore")

# Configuration


In [ ]:
prj_path = "../"
data_path = prj_path + "data/new_data/DH/squeezed/"
prj_path_opt = prj_path + "optimize_hyperparam/opt_results/"
output_process = prj_path + "data/new_data/DH/processed_data/"
output_featureselection = prj_path + "data/new_data/DH/feature_selection/"

In [ ]:
all_cities = [
    "An Giang",
    "BR Vũng Tàu",
    "Bình Phước",
    "Bình Thuận",
    "Bình Định",
    "Bạc Liêu",
    "Bắc Kạn",
    "Bắc Giang",
    "Cao Bằng",
    "Cà Mau",
    "Cần Thơ",
    "Gia Lai",
    "Hà Giang",
    "Hà Nội",
    "Hà Tĩnh",
    "Hòa Bình",
    "Hưng Yên",
    "Hải Dương",
    "Hải Phòng",
    "Khánh Hòa",
    "Kiên Giang",
    "Kon Tum",
    "Lai Châu",
    "Long An",
    "Lào Cai",
    "Lâm Đồng",
    "Lạng Sơn",
    "Nam Định",
    "Nghệ An",
    "Ninh Bình",
    "Ninh Thuận",
    "Phú Thọ",
    "Phú Yên",
    "Quảng Bình",
    "Quảng Nam",
    "Quảng Ngãi",
    "Quảng Ninh",
    "Quảng Trị",
    "Sóc Trăng",
    "Sơn La",
    "TT Huế",
    "Thanh Hóa",
    "Thái Bình",
    "Thái Nguyên",
    "Tiền Giang",
    "Trà Vinh",
    "Tuyên Quang",
    "Tây Ninh",
    "Vĩnh Phúc",
    "Yên Bái",
    "Điện Biên",
    "Đà Nẵng",
    "Đắk Nông",
    "Đắk Lắk",
    "Đồng Tháp",
]
cities = ["An Giang", "BR Vũng Tàu", "Bình Phước"]

In [ ]:
# Set hyperparameters as args using the Configuration class
class Configuration:
    def __init__(self):
        # lấy bộ test dài 36 tháng = 3 năm
        self.test_size = 36
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng
        self.look_back = 3
        # dự phóng n-step trong 6 tháng
        self.n_predicted_period_months = 6
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        self.device = torch.device("cuda")
        # self.device = torch.device("cpu")
        self.epochs = 300
        # others
        self.labels = "Dengue_fever_rates"
        # Input param for Optimize Run
        self.ntry = 1
        self.njob = 1


args = Configuration()

# Seeding


In [ ]:
def seed_everything(seed: int):
    import random
    import numpy as np
    import torch

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)


seed_everything(args.seed)

# Supporting functions


In [ ]:
import requests


def send_to_telegram(message):
    apiToken = "5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q"
    chatID = "@ptn_announcement"
    apiURL = f"https://api.telegram.org/bot{apiToken}/sendMessage"

    try:
        response = requests.post(apiURL, json={"chat_id": chatID, "text": message})
        print(response.text)
    except Exception as e:
        message_error = "Bị lỗi rùi: " + str(e)
        response = requests.post(
            apiURL, json={"chat_id": chatID, "text": message_error}
        )
        print(e)

In [ ]:
def get_dict_all_city_data():
    cities_data = {}
    for city in cities:
        city_result = pd.read_excel(
            prj_path + "data/new_data/DH/squeezed/squeezed_" + city + ".xlsx"
        )
        """Get all data from all city in 1997 - 2016"""
        city_result = city_result.loc[city_result["year_month"] < "2017-1-1"]
        cities_data[city] = city_result
    return cities_data

In [ ]:
# Define data (pre-)processing functions
# modification
def get_city_data(city_name, dict_full_data):
    """Returns Dengue fever rate and climate data"""
    city_data = dict_full_data[city_name].drop(
        columns=[
            "Diarrhoea_cases",
            "Diarrhoea_rates",
            "province",
            "Influenza_rates",
            "Influenza_cases",
            "Dengue_fever_cases",
            "year",
            "month",
        ],
        axis=1,
        inplace=False,
    )
    return city_data


def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != "Dengue_fever_rates":
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data


def impute_missing_value(city_data):
    """
    Imputes 0 for first 12 months,
    last year's value for months 12-24,
    and minimum value of last two years for months 25+
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(
                        city_data[col].iloc[index - 12], city_data[col].iloc[index - 24]
                    )
    return city_data

In [ ]:
def clean_full_data(dict_full_data):
    climate_and_disease_feats = [
        "Total_Evaporation",
        "Total_Rainfall",
        "Max_Daily_Rainfall",
        "n_raining_days",
        "Average_temperature",
        "Max_Average_Temperature",
        "Min_Average_Temperature",
        "Max_Absolute_Temperature",
        "Min_Absolute_Temperature",
        "Average_Humidity",
        "Min_Humidity",
        "n_hours_sunshine",
        "Dengue_fever_rates",
    ]
    for city in cities:
        city_data = get_city_data(city_name=city, dict_full_data=dict_full_data)
        city_data_features = city_data[climate_and_disease_feats]
        city_data_features = impute_missing_value(city_data_features)
        city_data_features = convert_to_stationary(city_data_features)
        city_data_features.dropna(inplace=True)
        city_data_features.loc[:, "year_month"] = city_data["year_month"]
        dict_full_data[city] = city_data_features
    return dict_full_data

In [ ]:
def split_data(data, look_back, n_nextstep=args.n_predicted_period_months):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]
    test = data[-args.test_size - look_back - (n_nextstep - 1) :]
    return train, test

In [ ]:
def to_supervised(data, d_out, d_in):
    """
    Frames time-series as supervised learning dataset.

    Args:
      d_in: lookback window
      d_out: number of predicted months

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            X.append(data[index:in_end, :-1])
            y.append(data[in_end:out_end, -1])
    return np.array(X), np.array(y)

In [ ]:
def getDataWithSelectedFeature(city, next_predicted_month):
    selected_feature = []
    df = pd.read_csv(
        output_featureselection
        + str(next_predicted_month)
        + "step_feature_selection_3_most.csv"
    )
    for row in range(len(df)):
        if df["City"][row] == city:
            selected_feature.append(df["1st_Feature"][row])
            selected_feature.append(df["2nd_Feature"][row])
            selected_feature.append(df["3rd_Feature"][row])
    return selected_feature

In [ ]:
def train_and_evaluate(df_full, model, location, feature_list, labels, scaler):
    """
    $df: pandas.DataFrame object containing data for training and testing model:
    $model: darts model object
    $feature_list: Names of the features used as model input
    $label: the value the model will be trained to predict
    $scaler: scaler object. Note: the scaler will be fitted on training data and applied to test data
    $lags: how much to look back into the past to output prediction
    $split_index: the point at which to divide train and test_data

    """
    df_train, df_eval = split_data(
        df_full, args.look_back, args.n_predicted_period_months
    )

    print("🍋🍋🍋🍋🍋Check var feature selectio n: ",feature_list)
    x_train = TimeSeries.from_dataframe(df_train, "year_month", feature_list)
    y_train = TimeSeries.from_dataframe(df_train, "year_month", labels)
    x_test = TimeSeries.from_dataframe(df_eval, "year_month", feature_list)
    y_test = TimeSeries.from_dataframe(df_eval, "year_month", labels)

    # Cách một khoảng phần này cho x_train thì mới đưa đc chuỗi x_test vừa đủ để dự đoán 36 lần 6step
    # ValueError: For the given forecasting horizon `n=xxx`, the provided
    # past covariates at dataset index `0` do not extend far enough into
    # the future. As `n > output_chunk_length` the past covariates must
    # start at time step `xxxxxx`, whereas now they start at
    # time step `xxxxx`.

    x_train_back = x_train[: -(args.n_predicted_period_months - 1)]
    y_train_back = y_train[: -(args.n_predicted_period_months - 1)]

    predict_list = []
    for i in range(args.test_size + (args.n_predicted_period_months - 1)):
        model.fit(y_train_back, past_covariates=x_train_back)
        prediction = model.predict(
            6,
            past_covariates=x_test[0 : i + args.look_back],
            num_samples=1,
            verbose=False,
        )
        x_train_back = x_train_back.append(x_test[i + 3])
        predict_list.append(np.array(prediction._xa).squeeze())
    y_pred_list = []
    for step in range(args.n_predicted_period_months):
        moving = args.n_predicted_period_months - 1 - step
        y_pred_list.append(
            [x[step] for x in predict_list][moving : args.test_size + moving]
        )

    df_eval_true_inverse = df_full[-args.test_size :]
    y_true = scaler.inverse_transform(df_eval_true_inverse.iloc[:, :-1])[
        :, [-1]
    ].reshape(args.test_size)

    y_pred_inverse_list = []
    for step in range(args.n_predicted_period_months):
        df_eval_pred_inverse = df_full[-args.test_size :]
        df_eval_pred_inverse[args.labels] = y_pred_list[step]  # step 1
        y_pred_inverse = scaler.inverse_transform(df_eval_pred_inverse.iloc[:, :-1])[
            :, [-1]
        ].reshape(args.test_size)
        y_pred_inverse_list.append(y_pred_inverse)

    y_pred_inverse_list
    df_compare_test_predict = pd.DataFrame(
        {
            "y_true": y_true,
            "y_pred_1step": y_pred_inverse_list[0],
            "y_pred_2step": y_pred_inverse_list[1],
            "y_pred_3step": y_pred_inverse_list[2],
            "y_pred_4step": y_pred_inverse_list[3],
            "y_pred_5step": y_pred_inverse_list[4],
            "y_pred_6step": y_pred_inverse_list[5],
        }
    )
    df_compare_test_predict.plot()
    plt.legend()
    plt.show()
    mse_nstep = []
    mae_nstep = []
    rmse_nstep = []
    mape_nstep = []
    for step in range(args.n_predicted_period_months):
        mse_nstep.append(mean_squared_error(y_true, y_pred_inverse_list[step]))
        mae_nstep.append(mean_absolute_error(y_true, y_pred_inverse_list[step]))
        rmse_nstep.append(mse_nstep[step] ** 0.5)
        mape_nstep.append(
            mean_absolute_percentage_error(y_true, y_pred_inverse_list[step])
        )
    return (
        model,
        y_true,
        y_pred_inverse_list,
        mse_nstep,
        mae_nstep,
        rmse_nstep,
        mape_nstep,
    )

In [ ]:
def output_prediction_for_location(
    df_full, model, location, feature_list, labels, scaler
):
    """train and generate prediction for a province
    df: DataFrame object containing features and label(s) for training model
    localtion: location_name
    feature_list: list of features used as model input,  must be among the column names of df
    labels: the values model will be trained to predict
    scaler: sklearn scaler object
    lags: how long into the past to look back when making prediction
    split_index: the point at which to divide data into the train and test subsets.
    """
    (
        model,
        y_true,
        y_pred_inverse_list,
        mse_nstep,
        mae_nstep,
        rmse_nstep,
        mape_nstep,
    ) = train_and_evaluate(df_full, model, location, feature_list, labels, scaler)

    df_prediction = pd.DataFrame(
        {
            "Date": df_full["year_month"][-args.test_size :],
            "Observed": y_true[-args.test_size :],
            f"{1}-month": y_pred_inverse_list[0],
            f"{2}-month": y_pred_inverse_list[1],
            f"{3}-month": y_pred_inverse_list[2],
            f"{4}-month": y_pred_inverse_list[3],
            f"{5}-month": y_pred_inverse_list[4],
            f"{6}-month": y_pred_inverse_list[5],
        }
    )

    df_prediction["City"] = location
    for step in range(args.n_predicted_period_months):
        df_prediction[f"RMSE_{step+1}-month"] = rmse_nstep[step]
        df_prediction[f"MAE_{step+1}-month"] = mae_nstep[step]
        df_prediction[f"MAPE_{step+1}-month"] = mape_nstep[step]
        df_prediction[f"MSE_{step+1}-month"] = mse_nstep[step]
    print("⭐️⭐️⭐️⭐️⭐️⭐️⭐️")
    display(df_prediction.head(5))
    print(mean(mae_nstep))
    return mean(mae_nstep)

In [ ]:
# Define Pytorch LSTM model
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM, self).__init__()
        self.lstm = nn.LSTM(
            input_size=n_feature,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,
        )
        self.linear = nn.Linear(hidden_size, args.n_predicted_period_months)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_period_months))

    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = output[:, -1, :]
        y_predicted = self.linear(last_hidden_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)


# Define Pytorch LSTM-ATT model
class MultiVariateLSTM_Attention(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(
            input_size=n_feature,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,
        )
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        # self.linear = nn.Linear(hidden_size*2, args.n_predicted_period_months)
        self.linear = nn.Linear(hidden_size, args.n_predicted_period_months)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_period_months))

    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = last_hidden[-1]
        remain_hidden_vector = output
        e_t = remain_hidden_vector.bmm(
            self.attention_linear(last_hidden_vector).unsqueeze(2)
        ).squeeze(-1)
        alpha_t = F.softmax(e_t, dim=1)
        attenion_vector = (
            remain_hidden_vector.transpose(2, 1).bmm(alpha_t.unsqueeze(2)).squeeze(-1)
        )
        # combine_vector = torch.cat((last_hidden_vector, attenion_vector), dim=1)
        # combine_vector = last_hidden_vector + attenion_vector
        y_predicted = self.linear(attenion_vector)
        if y_batch != None:
            # assert y_predicted.size() == y_batch.size()
            print("🍤🍤🍤🍤🍤🍤🍤🍤🍤🍤🍤")
            print("SHAPE y_predicted ", y_predicted.shape)
            print("SHAPE y_batch ", y_batch.shape)

            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)


# Define Pytorch CNN model
class MultivariateCNN(nn.Module):
    def __init__(self, num_filters=[100, 100, 100], dropout=0.01):
        super(MultivariateCNN, self).__init__()
        self.loss_fn = loss = nn.MSELoss()
        self.filter_sizes = [1, 2, 3]
        self.conv1d_list = nn.ModuleList(
            [
                nn.Conv1d(args.n_features, num_filters[i], self.filter_sizes[i])
                for i in range(len(self.filter_sizes))
            ]
        )
        self.linear = nn.Linear(np.sum(num_filters), args.n_predicted_period_months)
        self.dropout = nn.Dropout(dropout)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_period_months))

    def forward(self, X_batch, y_batch=None):
        X_batch = X_batch.permute(0, 2, 1)  # (batch_size, n_features, n_look_back)
        X_conv_list = [F.relu(conv1d(X_batch)) for conv1d in self.conv1d_list]
        X_pool_list = [
            F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in X_conv_list
        ]
        X_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in X_pool_list], dim=1)
        y_predicted = self.linear(self.dropout(X_fc))
        if y_batch != None:
            # assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)


# Define Pytorch Transformer model
class PositionalEncoder(nn.Module):
    def __init__(self, d_model=3, n_feature=3, dropout=0.1):
        super().__init__()

        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(n_feature, d_model)
        for pos in range(n_feature):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** (2 * i / d_model)))
                if i + 1 < d_model:
                    pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * i + 1) / d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x * math.sqrt(self.d_model)
        length = x.size(1)
        pe = Variable(self.pe[:, :length], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        x = self.dropout(x)

        return x


class TransformerModel(nn.Module):
    def __init__(self, d_input=3, n_head=3, hidden_size=256, n_layers=3, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.hidden_size = hidden_size
        self.pe = PositionalEncoder(dropout=dropout)
        encoder_layers = nn.TransformerEncoderLayer(
            d_model=d_input,
            nhead=n_head,
            dim_feedforward=hidden_size,
            dropout=dropout,
            activation="gelu",
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.decoder = nn.Linear(d_input * n_head, args.n_predicted_period_months)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_period_months))
        self.loss_fn = nn.MSELoss()

    def forward(self, X_batch, y_batch=None):
        X_batch = self.pe(X_batch)
        X_batch = self.transformer_encoder(X_batch)
        X_batch = X_batch.view(X_batch.size(0), -1)

        y_predicted = self.decoder(X_batch)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
        else:
            return y_predicted
        return X_batch

    def predict(self, X):
        with torch.no_grad():
            X = torch.tensor(X, device=args.device)
        return self.forward(X).squeeze()

In [ ]:
# Create class to train and evaluate models
class Trainer:
    def __init__(
        self,
        model_type,
        learning_rate,
        important_features,
        train_loader,
        test_tensor,
        n_layers=2,
        hidden_size=128,
        num_filters=[100, 100, 100],
        dropout=0.01,
        look_back="",
    ):
        """
        Initialise trainer, allowing input of LSTM, LSTM-ATT, or CNN
        hyperparameters. Adam optimiser used for all models.
        """
        self.model_type = model_type
        self.look_back = look_back
        self.model = self.init_model(
            model_type, n_layers, hidden_size, num_filters, dropout
        )
        self.model.double().to(args.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.important_features, self.train_loader, self.test_tensor = (
            important_features,
            train_loader,
            test_tensor,
        )

    def init_model(self, model_type, n_layers, hidden_size, num_filters, dropout):
        """Initialise a model based on whether LSTM, LSTM-ATT, or CNN is chosen."""
        print("🌷🌷🌷🌷🌷🌷🌷🌷🌷")
        print(model_type)
        if model_type == "LSTM":
            model = MultiVariateLSTM(args.n_features, n_layers, hidden_size)
        elif model_type == "LSTM_ATT":
            model = MultiVariateLSTM_Attention(args.n_features, n_layers, hidden_size)
        elif model_type == "CNN":
            model = MultivariateCNN(num_filters, dropout)
        elif model_type == "TRANSFORMERS":
            model = TransformerModel(
                d_input=args.look_back,
                n_head=3,
                hidden_size=hidden_size,
                n_layers=n_layers,
                dropout=dropout,
            )
        return model

    def step(self, batch):
        self.model.train()
        X_batch, y_batch = tuple(t.to(args.device) for t in batch)
        self.optimizer.zero_grad()
        y_pred, loss = self.model.forward(X_batch, y_batch)
        loss.backward()
        self.optimizer.step()
        return loss.mean().item()

    def validation(self):
        self.model.eval()
        eval_loss = 0.0

        result = {}

        y_true = np.array([])
        y_pred = np.array([])

        X_batch, y_batch = tuple(t.to(args.device) for t in self.test_tensor)
        with torch.no_grad():
            outputs, loss = self.model.forward(X_batch, y_batch)
            eval_loss = loss.mean().item()

        return eval_loss

    def train(self, epochs=20):
        print("🍭🍭🍭🍭🍭")
        best_lost = float("inf")
        best_model = None
        for epoch in range(epochs):
            total_loss = 0.0
            for batch in self.train_loader:
                loss = self.step(batch)
                total_loss += loss
            train_loss = total_loss / len(self.train_loader)
            eval_loss = self.validation()
            if eval_loss < best_lost:
                best_lost = eval_loss
                best_model = copy.deepcopy(self.model)
            if (epoch + 1) == epochs or (epoch + 1) in [
                c + 1 for c in range(epochs) if c % int(epochs / 4) == 0
            ]:
                print(
                    f"Epoch: {epoch:2}/{epochs:2} - train_loss: {train_loss:.4f} - test_loss: {eval_loss:4f}"
                )
        self.model = best_model
        self.model.eval()
        return None

    def evaluate_model(
        self,
        np_data=None,
        df_eval=None,
        plot=True,
        scaled=True,
        city=None,
        k_steps=None,
        scaler=None,
    ):
        assert scaled, "data must be scaled"
        self.model.eval()
        tensor_data = torch.from_numpy(np_data)  # np_data = bộ dữ liệu đưa vào
        rmse_list = []
        mae_list = []
        mape_list = []

        y_predicted_list = []
        y_true_list = []

        df_eval_true_inverse = df_eval[-args.test_size :]
        for k_steps in range(1, args.n_predicted_period_months + 1):
            y_predicted = []
            for index in range(tensor_data.size(0) - args.look_back):
                X = tensor_data[index : index + args.look_back, [0, 1, 2]]

                yhat = self.model.predict(X.unsqueeze(0))
                yhat = yhat.squeeze()

                y_predicted.append(yhat.detach().cpu().numpy()[k_steps - 1])

            moving = args.n_predicted_period_months - k_steps
            y_predicted = y_predicted[moving : args.test_size + moving]

            df_eval_pred_inverse = df_eval[-args.test_size :]
            df_eval_pred_inverse[args.labels] = y_predicted  # step 1
            y_predicted = scaler.inverse_transform(df_eval_pred_inverse.iloc[:, :-1])[
                :, [-1]
            ].reshape(args.test_size)
            y_true = scaler.inverse_transform(df_eval_true_inverse.iloc[:, :-1])[
                :, [-1]
            ].reshape(args.test_size)

            if plot == True:
                plt.plot(y_predicted, label="predicted")
                plt.plot(y_true, label="actual")
                plt.title(f"k-steps = {k_steps}")
                plt.legend()
                plt.show()

            rmse = mean_squared_error(y_true, y_predicted, squared=False)
            mae = mean_absolute_error(y_true, y_predicted)
            mape = mean_absolute_percentage_error(y_true, y_predicted)

            rmse_list.append(rmse)
            mae_list.append(mae)
            mape_list.append(mape)

            y_predicted_list.append(y_predicted)
            y_true_list.append(y_true)
        print("🍔🍔🍔🍔y_true_list🍔🍔🍔")
        print(y_true_list)
        return y_true_list, y_predicted_list, rmse_list, mae_list, mape_list

# Objective and Suggest Hyperparams of Darts Models


In [ ]:
def objective(model_name, trial, city):
    specific_data = pd.read_csv(
        output_process + city + "_train_preprocessed.csv",
        parse_dates=True,
        index_col=None,
        encoding="unicode_escape",
    )
    scaler = joblib.load(output_process + city + "_train_scalerMinMaxNorm.save")  # ok
    selected_features = getDataWithSelectedFeature(city, 6)
    df_train, df_eval = split_data(
        specific_data, args.look_back, args.n_predicted_period_months
    )
    train = df_train[selected_features + [args.labels]].iloc[:, :].to_numpy()
    test = df_eval[selected_features + [args.labels]].iloc[:, :].to_numpy()

    train_X, train_y = to_supervised(
        train, d_out=args.n_predicted_period_months, d_in=args.look_back
    )
    test_X, test_y = to_supervised(
        test, d_out=args.n_predicted_period_months, d_in=args.look_back
    )

    print("🍤🍤🍤🍤🍤🍤🍤🍤🍤🍤🍤")
    print("SHAPE train_y ", train_y.shape)
    print("SHAPE train_X ", train_X.shape)
    print("SHAPE test_y ", test_y.shape)
    print("SHAPE test_X ", test_X.shape)

    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=args.batch_size, shuffle=True)

    print("🍉🍉🍉🍉🍉🍉🍉🍉🍉🍉🍉🍉🍉", model_name)
    if model_name == "CNN":
        # Define search parameters
        args.epochs = trial.suggest_int("Epochs", 100, 500, step=10)
        learning_rate = trial.suggest_loguniform("Learning rate", 1e-4, 1e-2)
        num_filters = trial.suggest_categorical(
            "Num. filters",
            [
                [64, 64, 64],
                [100, 100, 100],
                [128, 128, 128],
                [16, 32, 64],
                [32, 64, 128],
            ],
        )
        dropout = trial.suggest_uniform("Dropout rate", 0.01, 0.80)
        # CNN model
        trainer = Trainer(
            model_type="CNN",
            learning_rate=learning_rate,
            important_features=selected_features,
            train_loader=train_loader,
            test_tensor=test_tensor,
            num_filters=num_filters,
            dropout=dropout,
        )
    elif model_name == "LSTM":
        args.epochs = trial.suggest_int("Epochs", 100, 500, step=10)
        hidden_size = trial.suggest_int("Hidden size", 128, 512, log=True)
        n_layers = trial.suggest_int("n layers", 2, 5)
        learning_rate = trial.suggest_loguniform("Learning rate", 1e-4, 1e-2)
        trainer = Trainer(
            model_type="LSTM",
            learning_rate=learning_rate,
            important_features=selected_features,
            train_loader=train_loader,
            test_tensor=test_tensor,
            n_layers=n_layers,
            hidden_size=hidden_size,
            look_back=args.look_back,
        )
    elif model_name == "LSTM_ATT":
        args.epochs = trial.suggest_int("Epochs", 100, 500, step=10)
        hidden_size = trial.suggest_int("Hidden size", 128, 512, log=True)
        n_layers = trial.suggest_int("n layers", 2, 10)
        learning_rate = trial.suggest_loguniform("Learning rate", 1e-4, 1e-2)
        trainer = Trainer(
            model_type="LSTM_ATT",
            learning_rate=learning_rate,
            important_features=selected_features,
            train_loader=train_loader,
            test_tensor=test_tensor,
            n_layers=n_layers,
            hidden_size=hidden_size,
            look_back=args.look_back,
        )

    elif model_name == "TRANSFORMERS":
        n_layers = trial.suggest_int("n layers", 2, 10)  # a
        hidden_size = trial.suggest_int("Hidden size", 128, 512, log=True)
        learning_rate = trial.suggest_loguniform("Learning rate", 1e-4, 1e-2)
        dropout = trial.suggest_uniform("Dropout rate", 0.01, 0.80)
        args.epochs = trial.suggest_int("Epochs", 100, 500, step=10)

        trainer = Trainer(
            model_type="TRANSFORMERS",
            important_features=selected_features,
            train_loader=train_loader,
            test_tensor=test_tensor,
            n_layers=n_layers,
            hidden_size=hidden_size,
            learning_rate=learning_rate,
            dropout=dropout,
        )

    # Train model
    trainer.train(epochs=args.epochs)

    # Evaluate model
    y_true, y_pred, rmse_list, mae_list, mape_list = trainer.evaluate_model(
        np_data=test, df_eval=df_eval, plot=True, scaled=True, city=city, scaler=scaler
    )
    # _, _, rmse, mae, = trainer.evaluate_model(np_data=test, plot=False, scaled=True, city=city, y_scaler=y_scaler)

    return mean(mae_list)

In [ ]:
def concate_2_filter_str(listfilter=""):
    string_filter = ",".join(str(e) for e in listfilter)
    return string_filter

# Main run optimize and save


In [ ]:
#########################
# Main Cell for optimize
#########################
model_name_list = [
    # "CNN",
    "LSTM",
    # "LSTM_ATT",
    # "TRANSFORMERS",
]

dt_started = datetime.now()

# Input param for Optimize Run
ntry = args.ntry
njob = args.njob


# Lưu thông tin traceback study và error city trong quá trình optimize
l_study_city = {}
l_errCity = []


if __name__ == "__main__":
    for model_name in model_name_list:
        print(f"🔥🔥🔥🔥🔥🔥🔥🔥🔥{model_name}")
        for city_index in range(len(cities)):
            print(f"🔥🔥🔥🔥🔥🔥🔥🔥{cities[city_index]}")
            # Use Tree-structured Parzen Estimator sampler to minimise RMSE
            sampler = optuna.samplers.TPESampler()
            study = optuna.create_study(sampler=sampler, direction="minimize")

            # truyền multiple param vào trong biến trial
            obj_func = lambda trial: objective(model_name, trial, cities[city_index])

            try:
                # Optimise over 100 trials
                study.optimize(obj_func, n_trials=ntry, n_jobs=njob)

                # Print results
                print("Study statistics for : ")
                print("  Number of finished trials: ", len(study.trials))

                print("Best trial of city: ", cities[city_index])
                best_trial = study.best_trial
                print("  Value: ", best_trial.value)
                if model_name == "CNN":
                    # lưu best param vào trong biến toàn cục
                    one_city_param = pd.DataFrame(
                        {
                            "City": [cities[city_index]],
                            "Alg_name": "CNN",
                            "Best_value": best_trial.value,
                            "n_try_opt": ntry,
                            "n Feature": args.n_features,  # dùng chung cho tất cả các model và algorithm
                            "Batch Size": args.batch_size,  # dùng chung cho tất cả các model và algorithm
                            "Lookback Window": args.look_back,  # dùng chung cho tất cả các tỉnh của CNN
                            "Epochs": best_trial.params["Epochs"],
                            "Hiden Size": "",  # CNN không quan tâm
                            "n Layers": "",  # CNN không quan tâm
                            "Learning rate": best_trial.params["Learning rate"],
                            "Num. filters": concate_2_filter_str(
                                best_trial.params["Num. filters"]
                            ),  # đoạn này ép kiểu không sẽ thành x 3 do cấu trúc []
                            "Dropout rate": best_trial.params["Dropout rate"],
                        },
                        index=[0],
                    )
                elif model_name == "LSTM":
                    # lưu best param vào trong biến toàn cục
                    one_city_param = pd.DataFrame(
                        {
                            "City": [cities[city_index]],
                            "Alg_name": "LSTM",
                            "Best_value": best_trial.value,
                            "n_try_opt": ntry,
                            "n Feature": args.n_features,  # dùng chung cho tất cả các model và algorithm
                            "Batch Size": args.batch_size,  # dùng chung cho tất cả các model và algorithm
                            "Lookback Window": args.look_back,
                            "Epochs": best_trial.params["Epochs"],
                            "Hiden Size": best_trial.params["Hidden size"],
                            "n Layers": best_trial.params["n layers"],
                            "Learning rate": best_trial.params["Learning rate"],
                            "Num. filters": "",  # đoạn này ép kiểu không sẽ thành x 3 do cấu trúc []
                            "Dropout rate": "",
                        },
                        index=[0],
                    )
                elif model_name == "LSTM_ATT":
                    one_city_param = pd.DataFrame(
                        {
                            "City": [cities[city_index]],
                            "Alg_name": "LSTM_ATT",
                            "Best_value": best_trial.value,
                            "n_try_opt": ntry,
                            "n Feature": args.n_features,  # dùng chung cho tất cả các model và algorithm
                            "Batch Size": args.batch_size,  # dùng chung cho tất cả các model và algorithm
                            "Lookback Window": args.look_back,
                            "Epochs": best_trial.params["Epochs"],
                            "Hiden Size": best_trial.params["Hidden size"],
                            "n Layers": best_trial.params["n layers"],
                            "Learning rate": best_trial.params["Learning rate"],
                            "Num. filters": "",
                            "Dropout rate": "",
                        },
                        index=[0],
                    )
                elif model_name == "TRANSFORMERS":
                    one_city_param = pd.DataFrame(
                        {
                            "City": [cities[city_index]],
                            "Alg_name": "transformer",
                            "Best_value": best_trial.value,
                            "n_try_opt": ntry,
                            "n Feature": args.n_features,  # dùng chung cho tất cả các model và algorithm
                            "Batch Size": args.batch_size,  # dùng chung cho tất cả các model và algorithm
                            "Lookback Window": args.look_back,
                            "Epochs": best_trial.params["Epochs"],
                            "Hidden Size": best_trial.params["Hidden size"],
                            "n Layers": best_trial.params["n layers"],
                            "Learning rate": best_trial.params["Learning rate"],
                            "Num. filters": "",  # Transformer không dùng
                            "Dropout rate": best_trial.params["Dropout rate"],
                        },
                        index=[0],
                    )
                folder_path = f"opt_results/opt_res_ml_26102023/{model_name}/"
                file_path = (
                    folder_path
                    + f"261023_DF_opt_hyperparam_{model_name}_multi-nstep.xlsx"
                )
                if os.path.isfile(file_path):
                    with pd.ExcelWriter(
                        file_path,
                        mode="a",
                        engine="openpyxl",
                        if_sheet_exists="overlay",
                    ) as writer:
                        one_city_param.to_excel(
                            writer, header=None, startrow=city_index + 1, index=False
                        )
                else:
                    if not (os.path.isdir(folder_path)):
                        os.mkdir(folder_path)
                    with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
                        one_city_param.to_excel(
                            writer, startrow=city_index, index=False
                        )
            except:  # có error thì lưu vào l_errCity để check lại sau
                l_errCity.append(cities[city_index])
    # lưu kết quả vào file CNN
#   best_param.to_excel(prj_path_opt+'cnn/diarrhoea_opt_hyperparam_cnn.xlsx')

dt_ended = datetime.now()
print("kết thúc study trong:", round((dt_ended - dt_started).total_seconds() / 60))

In [ ]:
# send_to_telegram("Chạy xong optimize rùiii!!Vô check thuiii!!!" )